# Ner_Train 파일 만들기

In [4]:
import pandas as pd
import numpy as np
import os
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

def process():
    df_ner = pd.read_csv('ner_tag.csv', header=None) 
    df_ner.columns = ['word', 'tag', 'intent']
    
    df_intent = pd.read_csv('./models/intent/Intent_train_data.csv', header=None) 
    df_intent.columns = ['sentence', 'intent', 'intent_info']
    df_intent = df_intent.drop_duplicates()
    
    result = []
    sentence = []
    
    for intent in df_intent["sentence"]:
        result.append([])
        # First Line
        data = f'; {intent}'
        result[len(result) - 1].append(data)
        
        # Second Line
        idx = 0
        word = ""
        tag = ""
        for ner in df_ner['word']:
            if ner in intent:
                word = ner
                tag = df_ner['tag'][idx]
                break;
            idx += 1;
            
        data = f'${intent}'
        if word != "":
            data = data.replace(word, f'<{word}:{tag}>')
        result[len(result) - 1].append(data)
        
        # Third Line
        data = p.pos(intent)
        for put in data:
            if put[0] == word:
                words = f"{idx}\t{put[0]}\t{put[1]}\tB_{tag}"
            else:
                words = f"{idx}\t{put[0]}\t{put[1]}\tO"
            
            result[len(result) - 1].append(words)
    return result

result = process()

In [5]:
toText = ""

for data in result:
    for words in data:
        toText += words
        toText += "\n"
    toText += "\n"
    
toText

'; 빵종류?\n$<빵종류:MENU>?\n4\t빵종류\tNNG\tB_MENU\n4\t?\tSF\tO\n\n; 종류?\n$종류?\n261\t종류\tNNG\tO\n261\t?\tSF\tO\n\n; 음료종류?\n$<음료종류:MENU>?\n7\t음료종류\tNNG\tB_MENU\n7\t?\tSF\tO\n\n; 커피종류?\n$<커피종류:MENU>?\n8\t커피종류\tNNG\tB_MENU\n8\t?\tSF\tO\n\n; 식사종류?\n$<식사종류:MENU>?\n5\t식사종류\tNNG\tB_MENU\n5\t?\tSF\tO\n\n; 에이드종류?\n$<에이드종류:MENU>?\n6\t에이드\tNNG\tO\n6\t종류\tNNG\tO\n6\t?\tSF\tO\n\n; 사이드메뉴는 뭐 있어?\n$<사이드메뉴:MENU>는 뭐 있어?\n0\t사이드\tNNP\tO\n0\t메뉴\tNNP\tO\n0\t는\tJX\tO\n0\t뭐\tNP\tO\n0\t있\tVX\tO\n0\t어\tEF\tO\n0\t?\tSF\tO\n\n; 사이드 메뉴는 뭐 있어?\n$<사이드 메뉴:MENU>는 뭐 있어?\n1\t사이드\tNNP\tO\n1\t메뉴\tNNP\tO\n1\t는\tJX\tO\n1\t뭐\tNP\tO\n1\t있\tVX\tO\n1\t어\tEF\tO\n1\t?\tSF\tO\n\n; 빵종류는 뭐 있어?\n$<빵종류:MENU>는 뭐 있어?\n4\t빵종류\tNNG\tB_MENU\n4\t는\tJX\tO\n4\t뭐\tNP\tO\n4\t있\tVX\tO\n4\t어\tEF\tO\n4\t?\tSF\tO\n\n; 종류는 뭐 있어?\n$종류는 뭐 있어?\n261\t종류\tNNG\tO\n261\t는\tJX\tO\n261\t뭐\tNP\tO\n261\t있\tVX\tO\n261\t어\tEF\tO\n261\t?\tSF\tO\n\n; 음료종류는 뭐 있어?\n$<음료종류:MENU>는 뭐 있어?\n7\t음료종류\tNNG\tB_MENU\n7\t는\tJX\tO\n7\t뭐\tNP\tO\n7\t있\tVX\tO\n7\t어\tEF\tO\n7\t?\tSF\tO\n\

In [6]:
with open('./models/ner/ner_train.txt', 'w', encoding='utf-8') as f:
    f.write(toText)

In [1]:
import pandas as pd
import numpy as np
import os
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

def process():
    df_ner = pd.read_csv('ner_tag.csv', header=None) 
    df_ner.columns = ['word', 'tag', 'intent']
    
    df_intent = pd.read_csv('./models/intent/Intent_train_data.csv', header=None) 
    df_intent.columns = ['sentence', 'intent', 'intent_info']
    df_intent = df_intent.drop_duplicates()
    
    result = []
    sentence = []
    
    for intent in df_intent["sentence"]:
        result.append([])
        # First Line
        data = f'; {intent}'
        result[len(result) - 1].append(data)
        
        # Second Line
        idx = 0
        word = ""
        tag = ""
        for ner in df_ner['word']:
            if ner in intent:
                word = ner
                tag = df_ner['tag'][idx]
                break;
            idx += 1;
            
        data = f'${intent}'
        if word != "":
            data = data.replace(word, f'<{word}:{tag}>')
        result[len(result) - 1].append(data)
        
        # Third Line
        data = p.pos(intent)
        for put in data:
            if put[0] == word:
                words = f"{idx}\t{put[0]}\t{put[1]}\tB_{tag}"
            else:
                words = f"{idx}\t{put[0]}\t{put[1]}\tO"
            
            result[len(result) - 1].append(words)
    return result

result = process()

toText = ""

for data in result:
    for words in data:
        toText += words
        toText += "\n"
    toText += "\n"
    
toText

with open('ner_train.txt', 'w', encoding='utf-8') as f:
    f.write(toText)# 필요한 import 하기
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
from utils.Preprocess import Preprocess
import os

In [2]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'wb', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

# 전처리 모듈
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'user_dic.tsv'))

In [3]:
# ① 학습용 말뭉치 데이터를 불러옴
corpus = read_file(os.path.join('./models/ner', 'ner_train.txt'))

In [4]:
len(corpus)

61999

In [5]:
corpus[:20]

[[('1', '가락지빵', 'NNG', 'B_FOOD'),
  ('2', '주문', 'NNP', 'O'),
  ('3', '하', 'VV', 'O'),
  ('4', '고', 'EC', 'O'),
  ('5', '싶', 'VX', 'O'),
  ('6', '어요', 'EC', 'O')],
 [('1', '가락지빵', 'NNG', 'B_FOOD'),
  ('2', '먹', 'VV', 'O'),
  ('3', '고', 'EC', 'O'),
  ('4', '싶', 'VX', 'O'),
  ('5', '어요', 'EC', 'O')],
 [('1', '가락지빵', 'NNG', 'B_FOOD'),
  ('2', '하', 'VV', 'O'),
  ('3', '고', 'EC', 'O'),
  ('4', '싶', 'VX', 'O'),
  ('5', '어요', 'EC', 'O')],
 [('1', '가락지빵', 'NNG', 'B_FOOD'),
  ('2', '원', 'NNB', 'O'),
  ('3', '하', 'XSA', 'O'),
  ('4', '어요', 'EC', 'O')],
 [('1', '가락지빵', 'NNG', 'B_FOOD'),
  ('2', '어떻', 'VA', 'O'),
  ('3', '게', 'EC', 'O'),
  ('4', '하', 'VV', 'O'),
  ('5', '아야', 'EC', 'O'),
  ('6', '되', 'VV', 'O'),
  ('7', '요', 'EF', 'O'),
  ('8', '?', 'SF', 'O')],
 [('1', '가락지빵', 'NNG', 'B_FOOD'),
  ('2', '어떻', 'VA', 'O'),
  ('3', '게', 'EC', 'O'),
  ('4', '하', 'VV', 'O'),
  ('5', '아야', 'EC', 'O'),
  ('6', '되', 'VV', 'O'),
  ('7', '어', 'EC', 'O')],
 [('1', '가락지빵', 'NNG', 'B_FOOD'),
  ('2', '하', 'VV', 

In [6]:
# ② 위에서 불러온 말뭉치 데이터에서 단어(w[1])와 BIO 태그(w[3])만 불러와 학습용 데이터셋 생성

sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
    
    sentences.append(sentence)
    tags.append(bio_tag)



In [7]:
sentences[:10]

[['가락지빵', '주문', '하', '고', '싶', '어요'],
 ['가락지빵', '먹', '고', '싶', '어요'],
 ['가락지빵', '하', '고', '싶', '어요'],
 ['가락지빵', '원', '하', '어요'],
 ['가락지빵', '어떻', '게', '하', '아야', '되', '요', '?'],
 ['가락지빵', '어떻', '게', '하', '아야', '되', '어'],
 ['가락지빵', '하', '고', '싶', '은데'],
 ['가락지빵', '알리', '어', '주', '어'],
 ['가락지빵', '가르치', '어', '주', '어'],
 ['가락지빵', '하', '고', '싶', '다']]

In [8]:
tags[:10]

[['B_FOOD', 'O', 'O', 'O', 'O', 'O'],
 ['B_FOOD', 'O', 'O', 'O', 'O'],
 ['B_FOOD', 'O', 'O', 'O', 'O'],
 ['B_FOOD', 'O', 'O', 'O'],
 ['B_FOOD', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B_FOOD', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B_FOOD', 'O', 'O', 'O', 'O'],
 ['B_FOOD', 'O', 'O', 'O', 'O'],
 ['B_FOOD', 'O', 'O', 'O', 'O'],
 ['B_FOOD', 'O', 'O', 'O', 'O']]

In [9]:
print("샘플 크기 : \n", len(sentences))
print("0번 째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번 째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))

샘플 크기 : 
 61999
0번 째 샘플 단어 시퀀스 : 
 ['가락지빵', '주문', '하', '고', '싶', '어요']
0번 째 샘플 bio 태그 : 
 ['B_FOOD', 'O', 'O', 'O', 'O', 'O']
샘플 단어 시퀀스 최대 길이 : 168
샘플 단어 시퀀스 평균 길이 : 8.796238649010467


In [10]:
# ③ 토크나이저 정의
# 단어 시퀀스의 경우 Preprocess 객체에서 생성하기 때문에 예제에서는 BIO 태그용 Tokenizer 객체 생성
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)  # 나중에 texts_to_sequences() 등을 수행하기 전에 꼭


In [11]:
# 생성된 사전 리스트를 이용해 단어와 태그 사전의 크기를 정의

In [12]:
# 우선 확인
p.word_index

{'OOV': 1,
 '하': 2,
 '주문': 3,
 '시': 4,
 '분': 5,
 '가능': 6,
 '예약': 7,
 '주': 8,
 '어요': 9,
 '고': 10,
 '싶': 11,
 '어': 12,
 '모레': 13,
 '내일': 14,
 '변경': 15,
 'ㄹ': 16,
 '취소': 17,
 '요': 18,
 '이': 19,
 '아': 20,
 '되': 21,
 '게': 22,
 'ㄴ': 23,
 '나요': 24,
 '있': 25,
 '수': 26,
 '다음': 27,
 '어떻': 28,
 'ㄴ가요': 29,
 '에': 30,
 '금주': 31,
 '로': 32,
 '는': 33,
 '을': 34,
 '나': 35,
 'ㅂ니다': 36,
 '가요': 37,
 '시간': 38,
 '으로': 39,
 '오늘': 40,
 '은데': 41,
 '다': 42,
 '가': 43,
 '덮밥': 44,
 '금일': 45,
 '밥': 46,
 '음식': 47,
 'ㄹ까요': 48,
 '문': 49,
 '아야': 50,
 '습니다': 51,
 '바로': 52,
 '4': 53,
 '었': 54,
 '은': 55,
 '1': 56,
 'ㄹ게요': 57,
 '겠': 58,
 '도': 59,
 '삼': 60,
 '국수': 61,
 '진행': 62,
 '콜': 63,
 '빵': 64,
 '거': 65,
 '드리': 66,
 '메뉴': 67,
 '의': 68,
 'ㄹ께': 69,
 '15': 70,
 '20': 71,
 '17': 72,
 '지': 73,
 '받': 74,
 '18': 75,
 '라면': 76,
 '았': 77,
 '11': 78,
 '지금': 79,
 '달걀': 80,
 '는데': 81,
 '5': 82,
 '면': 83,
 '날짜': 84,
 '톡': 85,
 '12': 86,
 '담': 87,
 '를': 88,
 '8': 89,
 '16': 90,
 '짜': 91,
 '19': 92,
 '14': 93,
 '보': 94,
 '당일': 95,
 '아요'

In [13]:
tag_tokenizer.word_index

{'O': 1,
 'B_DT': 2,
 'B_FOOD': 3,
 'I': 4,
 'B_OG': 5,
 'B_PS': 6,
 'B_LC': 7,
 'NNP': 8,
 'B_TI': 9}

In [14]:
# 단어사전 및 태그 사전 크기
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

BIO 태그 사전 크기 : 10
단어 사전 크기 : 17752


In [15]:
# 학습용 단어 시퀀스 생성
x_train = [p.get_wordidx_sequence(sent) for sent in sentences]
y_train = tag_tokenizer.texts_to_sequences(tags)

index_to_ner = tag_tokenizer.index_word
index_to_ner[0] = 'PAD'

In [17]:
len(x_train)

61999

In [18]:
x_train[:10]

[[1, 3, 2, 10, 11, 9],
 [1, 233, 10, 11, 9],
 [1, 2, 10, 11, 9],
 [1, 99, 2, 9],
 [1, 28, 22, 2, 50, 21, 18, 1],
 [1, 28, 22, 2, 50, 21, 12],
 [1, 2, 10, 11, 41],
 [1, 135, 12, 8, 12],
 [1, 145, 12, 8, 12],
 [1, 2, 10, 11, 42]]

In [19]:
y_train[:10]

[[3, 1, 1, 1, 1, 1],
 [3, 1, 1, 1, 1],
 [3, 1, 1, 1, 1],
 [3, 1, 1, 1],
 [3, 1, 1, 1, 1, 1, 1, 1],
 [3, 1, 1, 1, 1, 1, 1],
 [3, 1, 1, 1, 1],
 [3, 1, 1, 1, 1],
 [3, 1, 1, 1, 1],
 [3, 1, 1, 1, 1]]

In [20]:
index_to_ner   # 0: 'PAD'  가 추가되었다

{1: 'O',
 2: 'B_DT',
 3: 'B_FOOD',
 4: 'I',
 5: 'B_OG',
 6: 'B_PS',
 7: 'B_LC',
 8: 'NNP',
 9: 'B_TI',
 0: 'PAD'}

In [21]:
# ⑤ 시퀀스 패딩 처리
max_len = 40   # 시퀀스 벡터 크기는 위에서 계산해본 평균 길이보다는 넉넉하게 40으로 설정
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)


In [22]:
x_train

array([[  1,   3,   2, ...,   0,   0,   0],
       [  1, 233,  10, ...,   0,   0,   0],
       [  1,   2,  10, ...,   0,   0,   0],
       ...,
       [  1,   1,   1, ...,   0,   0,   0],
       [  1,   1,   1, ...,   0,   0,   0],
       [  1,   1, 360, ...,   0,   0,   0]])

In [23]:
y_train

array([[3, 1, 1, ..., 0, 0, 0],
       [3, 1, 1, ..., 0, 0, 0],
       [3, 1, 1, ..., 0, 0, 0],
       ...,
       [2, 2, 3, ..., 0, 0, 0],
       [2, 2, 3, ..., 0, 0, 0],
       [2, 2, 3, ..., 0, 0, 0]])

In [24]:
x_train.shape, y_train.shape

((61999, 40), (61999, 40))

In [25]:
# ⑥ 학습 데이터와 테스트 데이터를 8:2의 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
                                                    test_size=.2,
                                                    random_state=1234)


In [26]:
len(x_train), len(x_test), len(y_train), len(y_test)

(49599, 12400, 49599, 12400)

In [28]:
tag_size

10

In [29]:
# 출력 데이터를 one-hot encoding
# ★ 한번만 실행할것!
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)  # tag_size. BIO 태그 사전 크기, 
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)    # 현재 tag_size 는 10 이다

In [30]:
y_train.shape, y_test.shape

((49599, 40, 10), (12400, 40, 10))

In [31]:
y_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [32]:
print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (49599, 40)
학습 샘플 레이블 형상 :  (49599, 40, 10)
테스트 샘플 시퀀스 형상 :  (12400, 40)
테스트 샘플 레이블 형상 :  (12400, 40, 10)


In [33]:
# ⑦ 모델 정의 (Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])

In [34]:
model_file = os.path.join(r'./models/ner', 'ner_model.h5')
model_file

'./models/ner\\ner_model.h5'

In [35]:
# 학습하기
model.fit(x_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
388/388 [==============================] - 130s 309ms/step - loss: 0.0266 - accuracy: 0.9650
Epoch 2/10
388/388 [==============================] - 118s 305ms/step - loss: 0.0086 - accuracy: 0.9871
Epoch 3/10
388/388 [==============================] - 119s 308ms/step - loss: 0.0059 - accuracy: 0.9910
Epoch 4/10
388/388 [==============================] - 119s 306ms/step - loss: 0.0047 - accuracy: 0.9926
Epoch 5/10
388/388 [==============================] - 119s 307ms/step - loss: 0.0040 - accuracy: 0.9938
Epoch 6/10
388/388 [==============================] - 119s 307ms/step - loss: 0.0035 - accuracy: 0.9944
Epoch 7/10
388/388 [==============================] - 119s 307ms/step - loss: 0.0032 - accuracy: 0.9949
Epoch 8/10
388/388 [==============================] - 119s 307ms/step - loss: 0.0030 - accuracy: 0.9954
Epoch 9/10
388/388 [==============================] - 121s 311ms/step - loss: 0.0029 - accuracy: 0.9953
Epoch 10/10
388/388 [==============================] - 131s 339m

In [36]:
print("평가 결과 : ", model.evaluate(x_test, y_test)[1])
model.save(os.path.join('./models/ner', 'ner_model.h5'))  # 학습 완료된 모델 저장

388/388 [==============================] - 10s 24ms/step - loss: 0.0134 - accuracy: 0.9841
평가 결과 :  0.9841488599777222


In [37]:
# F1 Score 계산

In [38]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):  # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
    result = []
    for sequence in sequences:  # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence:  # 시퀀스로부터 예측값을 하나씩 꺼낸다.
            pred_index = np.argmax(pred)  # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))  # 'PAD'는 'O'로 변경
        result.append(temp)
    return result

In [39]:
from seqeval.metrics import f1_score, classification_report

In [40]:
# 테스트 데이터셋의 NER 예측
# F1 score 를 게산하기 위해 모델의 predict() 함수를 통해 테스트용 데이터 셋의 결과를 예측합니다.
# 해당 함수의 입력은 시퀀스 번호로 인코딩 된 테스트 단어 시퀀스 (ndarray 배열)을 사용
# 해당 함수의 결과는 예측된 NER 태그 정보가 담긴 ndarray 배열을 리턴
y_predicted = model.predict(x_test)

In [41]:
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test)    # 실제 NER

In [42]:
# F1 평가 결과
# seqeval.metrics 모듈의 classification+_report 함수를 통해 NER 태그 별로 계산된 precision, recall 그리고 f1_score 값을 출력

print(classification_report(test_tags, pred_tags))

# f1_score() 함수 만으로 f1 score 값만 출력 가능
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_FOOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequ

              precision    recall  f1-score   support

          NP       1.00      1.00      1.00       303
           _       0.45      0.53      0.48       658
         _DT       1.00      1.00      1.00     13683
       _FOOD       1.00      1.00      1.00     11655
         _LC       0.73      0.57      0.64       314
         _OG       0.50      0.56      0.53       460
         _PS       0.55      0.56      0.56       396
         _TI       0.86      0.70      0.77        61

   micro avg       0.96      0.97      0.97     27530
   macro avg       0.76      0.74      0.75     27530
weighted avg       0.97      0.97      0.97     27530

F1-score: 96.6%
